This GAN is based on healthGAN description (https://github.com/yknot/ESANN2019)

In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

In [2]:
HOME_PATH = '' #home path of the project
TRAIN_FILE = 'REAL DATASETS/TRAIN DATASETS/F_IndianLiverPatient_Real_Train.csv'
SYNTHETIC_FILE = 'SYNTHETIC DATASETS/WGANGP/F_IndianLiverPatient_Synthetic_WGANGP.csv'

In [3]:
#define directory of functions and actual directory
FUNCTIONS_DIR = HOME_PATH + 'STDG APPROACHES'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(FUNCTIONS_DIR)

#import functions for univariate resemblance analisys
from preprocessing import DataPreProcessor

#change directory to actual directory
os.chdir(ACTUAL_DIR)

from ydata_synthetic.synthesizers.regular import WGAN_GP
print('Functions imported!!')

Functions imported!!


## Data Preprocessing

In [4]:
import pandas as pd
real_data = pd.read_csv(HOME_PATH + TRAIN_FILE)
cat_cols = ['gender','class']
for c in cat_cols :
    real_data[c] = real_data[c].astype('category')
data_cols = real_data.columns
data_train = real_data
real_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,68,Female,0.6,0.1,1620.0,95.0,127.0,4.6,2.1,0.80,1.0
1,31,Male,1.3,0.5,184.0,29.0,32.0,6.8,3.4,1.00,1.0
2,28,Male,0.8,0.3,190.0,20.0,14.0,4.1,2.4,1.40,1.0
3,60,Male,2.3,0.6,272.0,79.0,51.0,6.6,3.5,1.10,1.0
4,48,Female,0.9,0.2,173.0,26.0,27.0,6.2,3.1,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
461,75,Female,0.8,0.2,188.0,20.0,29.0,4.4,1.8,0.60,1.0
462,36,Male,5.3,2.3,145.0,32.0,92.0,5.1,2.6,1.00,2.0
463,37,Male,0.7,0.2,235.0,96.0,54.0,9.5,4.9,1.00,1.0
464,17,Female,0.5,0.1,206.0,28.0,21.0,7.1,4.5,1.70,2.0


In [5]:
# data configuration
preprocessor = DataPreProcessor(data_train)
data_train = preprocessor.preprocess_train_data()
data_train

,age,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,gender0,gender1,class0,class1
0,1.521598,-0.429526,-0.495232,5.429707,0.054658,0.033501,-1.751896,-1.338053,0.065629,1.798990,-1.798990,0.645301,-0.645301
1,-0.845149,-0.318923,-0.350905,-0.432725,-0.280949,-0.269359,0.273413,0.302490,0.065660,-0.555868,0.555868,0.645301,-0.645301
2,-1.037048,-0.397925,-0.423069,-0.408230,-0.326714,-0.326743,-2.212193,-0.959466,0.065721,-0.555868,0.555868,0.645301,-0.645301
3,1.009869,-0.160920,-0.314824,-0.073467,-0.026701,-0.208787,0.089294,0.428686,0.065675,-0.555868,0.555868,0.645301,-0.645301
4,0.242275,-0.382125,-0.459150,-0.477632,-0.296204,-0.285299,-0.278944,-0.076096,0.065660,1.798990,-1.798990,0.645301,-0.645301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,1.969361,-0.397925,-0.459150,-0.416395,-0.326714,-0.278923,-1.936015,-1.716639,0.065598,1.798990,-1.798990,0.645301,-0.645301
462,-0.525319,0.313091,0.298564,-0.591941,-0.265695,-0.078079,-1.291599,-0.707075,0.065660,-0.555868,0.555868,-1.549664,1.549664
463,-0.461352,-0.413725,-0.459150,-0.224518,0.059743,-0.199223,2.759019,2.195425,0.065660,-0.555868,0.555868,0.645301,-0.645301
464,-1.740675,-0.445326,-0.495232,-0.342910,-0.286034,-0.304427,0.549591,1.690642,0.065767,1.798990,-1.798990,-1.549664,1.549664


## Train the Model

Next, lets define the neural network for generating synthetic data. We will be using a [GAN](https://www.wikiwand.com/en/Generative_adversarial_network) network that comprises of an generator and discriminator that tries to beat each other and in the process learns the vector embedding for the data. 

The model was taken from a [Github repository](https://github.com/ydataai/gan-playground) where it is used to generate synthetic data on credit card fraud data. 

Next, lets define the training parameters for the GAN network.

In [10]:
# training configuration
noise_dim = 32
dim = 128
batch_size = 16

log_step = 200
epochs = 5000+1
learning_rate = 5e-4

beta_1 = 0.5
beta_2 = 0.9

models_dir = 'my_model_datasetF/'

data_dim = data_train.shape[1]
print('Shape of data: ', data_train.shape)

Shape of data:  (466, 13)


In [11]:
#Define the GAN and training parameters
gan_args = [batch_size, learning_rate, beta_1, beta_2, noise_dim, data_dim, dim]
train_args = ['', epochs, log_step]

Finally, let's run the training and see if the model is able to learn something.

In [12]:
!mkdir my_model_datasetF
!mkdir my_model_datasetF/gan
!mkdir my_model_datasetF/gan/saved

mkdir: cannot create directory ‘my_model_datasetF’: File exists
mkdir: cannot create directory ‘my_model_datasetF/gan’: File exists
mkdir: cannot create directory ‘my_model_datasetF/gan/saved’: File exists


In [13]:
#Training the GAN model chosen: Vanilla GAN, CGAN, DCGAN, etc.
synthesizer = WGAN_GP(gan_args, n_critic=2)
synthesizer.train(data_train, train_args)

  0%|          | 14/5001 [00:01<06:27, 12.87it/s] 

Iteration: 0 | disc_loss: 0.5279436707496643 | gen_loss: 0.07475227117538452
Iteration: 1 | disc_loss: 0.29112714529037476 | gen_loss: 0.022096257656812668
Iteration: 2 | disc_loss: 0.17718756198883057 | gen_loss: -0.05179682374000549
Iteration: 3 | disc_loss: 0.15675950050354004 | gen_loss: -0.11358264088630676
Iteration: 4 | disc_loss: 0.1792372465133667 | gen_loss: -0.1782449185848236
Iteration: 5 | disc_loss: -0.08790396898984909 | gen_loss: -0.24104702472686768
Iteration: 6 | disc_loss: -0.16795432567596436 | gen_loss: -0.18354544043540955
Iteration: 7 | disc_loss: -0.2687031030654907 | gen_loss: -0.12663710117340088
Iteration: 8 | disc_loss: -0.41209253668785095 | gen_loss: -0.016986941918730736
Iteration: 9 | disc_loss: -0.5620439648628235 | gen_loss: 0.059894584119319916
Iteration: 10 | disc_loss: -0.6104713082313538 | gen_loss: 0.11633134633302689
Iteration: 11 | disc_loss: -0.7155176997184753 | gen_loss: 0.1916421353816986
Iteration: 12 | disc_loss: -0.5749183893203735 | gen_

  1%|          | 40/5001 [00:01<01:57, 42.07it/s]

Iteration: 22 | disc_loss: -0.7092416882514954 | gen_loss: 0.07758018374443054
Iteration: 23 | disc_loss: -0.7484418153762817 | gen_loss: 0.12928205728530884
Iteration: 24 | disc_loss: -0.6933001279830933 | gen_loss: 0.08313816040754318
Iteration: 25 | disc_loss: -0.6640810966491699 | gen_loss: 0.0852566733956337
Iteration: 26 | disc_loss: -0.6532052755355835 | gen_loss: 0.0656396895647049
Iteration: 27 | disc_loss: -0.6369421482086182 | gen_loss: 0.10274341702461243
Iteration: 28 | disc_loss: -0.6829207539558411 | gen_loss: 0.12103349715471268
Iteration: 29 | disc_loss: -0.6627185344696045 | gen_loss: 0.07410874962806702
Iteration: 30 | disc_loss: -0.6326550841331482 | gen_loss: 0.03692933917045593
Iteration: 31 | disc_loss: -0.6896820068359375 | gen_loss: -0.0055745248682796955
Iteration: 32 | disc_loss: -0.733424961566925 | gen_loss: 0.06151959300041199
Iteration: 33 | disc_loss: -0.5342377424240112 | gen_loss: 0.20165683329105377
Iteration: 34 | disc_loss: -0.6293401122093201 | gen

  1%|▏         | 66/5001 [00:01<01:11, 69.12it/s]

...

Iteration: 4931 | disc_loss: -0.3495176434516907 | gen_loss: -0.06693359464406967
Iteration: 4932 | disc_loss: -0.17596697807312012 | gen_loss: 0.09698435664176941
Iteration: 4933 | disc_loss: -0.06403519958257675 | gen_loss: -0.07667580246925354
Iteration: 4934 | disc_loss: -0.18604691326618195 | gen_loss: -0.15715549886226654
Iteration: 4935 | disc_loss: -0.28788840770721436 | gen_loss: -0.02153274603188038
Iteration: 4936 | disc_loss: -0.2647229731082916 | gen_loss: -0.2261134684085846
Iteration: 4937 | disc_loss: -0.2474995255470276 | gen_loss: -0.03369247168302536
Iteration: 4938 | disc_loss: -0.19290770590305328 | gen_loss: -0.07593080401420593
Iteration: 4939 | disc_loss: -0.3051542341709137 | gen_loss: 0.03331426903605461
Iteration: 4940 | disc_loss: -0.1510629653930664 | gen_loss: -0.020596280694007874
Iteration: 4941 | disc_loss: -0.18292853236198425 | gen_loss: -0.033134639263153076
Iteration: 4942 | disc_loss: -0.17636272311210632 | gen_loss: 0.10261958837509155
Iteration: 

 99%|█████████▉| 4967/5001 [00:41<00:00, 113.52it/s]

Iteration: 4954 | disc_loss: -0.15418413281440735 | gen_loss: -0.03177223354578018
Iteration: 4955 | disc_loss: -0.2175995409488678 | gen_loss: -0.06984925270080566
Iteration: 4956 | disc_loss: -0.26684170961380005 | gen_loss: -0.17369160056114197
Iteration: 4957 | disc_loss: -0.2523361146450043 | gen_loss: -0.0894273966550827
Iteration: 4958 | disc_loss: -0.23669354617595673 | gen_loss: -0.04028835520148277
Iteration: 4959 | disc_loss: -0.18592718243598938 | gen_loss: -0.03955524042248726
Iteration: 4960 | disc_loss: -0.22568747401237488 | gen_loss: -0.11099366843700409
Iteration: 4961 | disc_loss: -0.20548883080482483 | gen_loss: -0.04685183987021446
Iteration: 4962 | disc_loss: -0.19093413650989532 | gen_loss: -0.018962297588586807
Iteration: 4963 | disc_loss: -0.09384218603372574 | gen_loss: -0.17347246408462524
Iteration: 4964 | disc_loss: -0.3338223397731781 | gen_loss: -0.20772618055343628
Iteration: 4965 | disc_loss: -0.2912767827510834 | gen_loss: -0.05914093554019928
Iteratio

100%|██████████| 5001/5001 [00:42<00:00, 118.43it/s]

Iteration: 4979 | disc_loss: -0.18023622035980225 | gen_loss: -0.028157634660601616
Iteration: 4980 | disc_loss: -0.24097058176994324 | gen_loss: 0.04423074424266815
Iteration: 4981 | disc_loss: -0.23303285241127014 | gen_loss: -0.08991800248622894
Iteration: 4982 | disc_loss: -0.08419256657361984 | gen_loss: -0.0423443429172039
Iteration: 4983 | disc_loss: -0.2762821614742279 | gen_loss: -0.008487751707434654
Iteration: 4984 | disc_loss: -0.24421025812625885 | gen_loss: -0.04474492371082306
Iteration: 4985 | disc_loss: -0.27353742718696594 | gen_loss: -0.07789111882448196
Iteration: 4986 | disc_loss: -0.22197426855564117 | gen_loss: -0.03580350801348686
Iteration: 4987 | disc_loss: -0.28345373272895813 | gen_loss: -0.16100819408893585
Iteration: 4988 | disc_loss: -0.3017561733722687 | gen_loss: -0.047414377331733704
Iteration: 4989 | disc_loss: -0.17838355898857117 | gen_loss: -0.1841980516910553
Iteration: 4990 | disc_loss: -0.22469981014728546 | gen_loss: -0.10221774131059647
Iterat

In [14]:
synthesizer.generator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(16, 32)]                0         
_________________________________________________________________
dense_8 (Dense)              (16, 128)                 4224      
_________________________________________________________________
dense_9 (Dense)              (16, 256)                 33024     
_________________________________________________________________
dense_10 (Dense)             (16, 512)                 131584    
_________________________________________________________________
dense_11 (Dense)             (16, 13)                  6669      
Total params: 175,501
Trainable params: 175,501
Non-trainable params: 0
_________________________________________________________________


In [15]:
synthesizer.critic.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(16, 13)]                0         
_________________________________________________________________
dense_12 (Dense)             (16, 512)                 7168      
_________________________________________________________________
dropout_2 (Dropout)          (16, 512)                 0         
_________________________________________________________________
dense_13 (Dense)             (16, 256)                 131328    
_________________________________________________________________
dropout_3 (Dropout)          (16, 256)                 0         
_________________________________________________________________
dense_14 (Dense)             (16, 128)                 32896     
_________________________________________________________________
dense_15 (Dense)             (16, 1)                   129 

## Generate data

In [16]:
size = len(data_train)
generated_samples = synthesizer.sample(size)
generated_samples.columns = data_train.columns
generated_samples

Synthetic data generation: 100%|██████████| 30/30 [00:00<00:00, 228.13it/s]


,age,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,gender0,gender1,class0,class1
0,-0.968457,-0.288383,-0.421187,-0.443180,-0.186732,-0.186278,1.082598,1.631181,0.040358,-0.546173,0.518191,0.680930,-0.608375
1,-0.209537,-0.106508,-0.180784,-0.129899,0.044517,-0.184117,0.863898,1.081858,0.054888,-0.530286,0.508825,-1.371416,1.405906
2,-0.641354,-0.259863,-0.326796,-0.275668,0.001210,-0.209180,1.061564,1.458473,0.030768,-0.572998,0.527597,-0.976692,0.984850
3,0.067705,-0.233378,-0.322822,-0.364250,-0.166107,-0.248514,0.621438,1.416883,0.060885,-0.527093,0.514541,-1.401724,1.429866
4,-0.291915,-0.097946,-0.165598,-0.113283,0.104390,-0.157887,0.964813,1.025213,0.059571,-0.508900,0.480830,-1.384795,1.412407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,-0.342521,-0.123632,-0.178721,-0.092311,0.116826,-0.164984,0.997155,1.102027,0.054709,-0.507491,0.495516,-1.363245,1.384906
12,-0.875445,-0.313375,-0.374917,-0.404709,-0.063845,-0.224272,1.128848,1.655192,0.020558,-0.540406,0.527076,-0.356913,0.376210
13,-0.704595,-0.177287,-0.259306,-0.231077,-0.055231,-0.148295,1.006025,1.285564,0.049293,-0.486796,0.494849,-0.353523,0.384079
14,-0.521921,-0.153495,-0.127791,0.936775,-0.185319,0.089437,0.435336,0.559908,0.096128,-0.518203,0.559129,0.697132,-0.590177


## Transform and process generated data

In [17]:
synthetic_data = preprocessor.transform_data(generated_samples)
synthetic_data = synthetic_data[0:len(real_data)]
synthetic_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,29,Male,1.493286,0.305215,181.439041,47.528675,58.060356,7.678981,4.452882,-164.402267,1.0
1,40,Male,2.644370,0.971491,258.177002,93.005783,58.738285,7.441417,4.017587,-69.416679,2.0
2,34,Male,1.673791,0.566819,222.470963,84.489075,50.876556,7.656133,4.316024,-227.091599,2.0
3,45,Male,1.841415,0.577833,200.772736,51.584827,38.538437,7.178044,4.283068,-30.214041,2.0
4,39,Male,2.698557,1.013579,262.246979,104.780251,66.965988,7.551036,3.972701,-38.801598,2.0
...,...,...,...,...,...,...,...,...,...,...,...
13,27,Male,1.151936,0.150563,171.930328,47.628956,44.985054,7.746896,4.513616,-256.732788,1.0
14,39,Male,2.686199,1.038701,263.196320,102.436165,61.872330,7.508008,3.969202,-129.229355,2.0
15,39,Male,2.645507,1.042506,276.282898,104.695000,66.229935,7.540930,3.992946,-187.846268,2.0
0,27,Male,1.079657,0.180837,171.397888,56.983002,41.766006,7.826374,4.611629,-310.726685,1.0


In [18]:
print(real_data.dtypes, '\n', synthetic_data.dtypes)
print(real_data.shape, synthetic_data.shape)

age           int64
gender     category
TB          float64
DB          float64
alkphos     float64
sgpt        float64
sgot        float64
TP          float64
ALB         float64
A_G         float64
class      category
dtype: object 
 age          int64
gender      object
TB         float64
DB         float64
alkphos    float64
sgpt       float64
sgot       float64
TP         float64
ALB        float64
A_G        float64
class      float64
dtype: object
(466, 11) (466, 11)


In [19]:
real_data.describe()

,age,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G
count,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000
mean,44.212446,3.318455,1.472532,289.995708,84.251073,116.491416,6.503004,3.160300,-428.230429
std,15.650069,6.335770,2.774472,245.212815,196.869782,314.013021,1.087422,0.793272,6544.232426
min,4.000000,0.400000,0.100000,63.000000,10.000000,11.000000,2.700000,0.900000,-100000.000000
25%,32.250000,0.800000,0.200000,174.000000,24.000000,26.000000,5.800000,2.600000,0.700000
50%,45.000000,1.000000,0.300000,206.000000,36.000000,43.000000,6.600000,3.100000,0.980000
75%,56.000000,2.675000,1.300000,298.000000,62.000000,88.000000,7.200000,3.800000,1.100000
max,85.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000


In [20]:
synthetic_data.describe()

,age,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
count,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000
mean,35.139485,2.078645,0.708242,240.919958,81.199831,60.184746,7.573658,4.178481,-179.052965,1.639485
std,5.411643,0.638313,0.357127,83.526800,21.165756,21.129761,0.192771,0.274893,126.802563,0.480666
min,23.000000,0.688439,-0.118866,145.981934,37.907944,18.565765,6.571207,3.167132,-526.067139,1.000000
25%,31.000000,1.544543,0.409270,199.516270,59.882494,48.069136,7.489009,3.985295,-268.500732,1.000000
50%,37.000000,2.277759,0.834267,244.749008,85.503040,57.696060,7.592279,4.093067,-173.894882,2.000000
75%,39.000000,2.613225,1.009911,264.475342,99.630133,66.842031,7.699375,4.375693,-96.811241,2.000000
max,47.000000,3.252332,1.514067,1213.167480,119.213799,195.690598,7.928803,4.863639,406.610077,2.000000


In [21]:
#Save generated samples
synthetic_data.to_csv(HOME_PATH + SYNTHETIC_FILE, index=False)